In [ ]:
import pandas
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification
from sklearn import datasets, linear_model
from yellowbrick.features import RFECV
import matplotlib.pyplot as plt
from sklearn.cross_validation import ShuffleSplit
from collections import defaultdict
from sklearn import cross_validation
#from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.feature_selection import GenericUnivariateSelect
from sklearn import datasets
#from sklearn.feature_selection import VarianceThreshold
from sklearn import model_selection
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
def preprocessData(data):
    data = data.replace('Unknown', np.nan)
    data.dropna(inplace=True)
    
    return data


In [ ]:
attribute = ['address', 'rooms', 'type', 'method', 'realestate_agent', 'date', 'distance', 'postcode', 'bedrooms',
             'bathrooms', 'car_parks', 'landsize', 'building_area', 'year_built',
            'council_area', 'lattitude', 'longtitude', 'region_name', 'suburb_property_count']
propertyData = pandas.read_csv("property_prices.csv")
propertyData = preprocessData(propertyData)

In [ ]:
propertyData['lowBand'], propertyData['highBand'] = propertyData['price_bands'].str.split('-', 1).str
propertyData['lowBand'] = propertyData['lowBand'].str[:-1]
propertyData['highBand'] = propertyData['highBand'].str[:-1]

for attr in attribute: 

    Y = propertyData['lowBand']
    X = propertyData.drop(columns=['id','lowBand', 'highBand','price_bands', attr])

In [ ]:
def test_f_classif():

    X, y = make_classification(n_samples=200, n_features=20,
                               n_informative=3, n_redundant=2,
                               n_repeated=0, n_classes=8,
                               n_clusters_per_class=1, flip_y=0.0,
                               class_sep=10, shuffle=False, random_state=0)

    F, pv = f_classif(X, Y)
    F_sparse, pv_sparse = f_classif(sparse.csr_matrix(X), Y)
    assert_true((F > 0).all())
    assert_true((pv > 0).all())
    assert_true((pv < 1).all())
    assert_true((pv[:5] < 0.05).all())
    assert_true((pv[5:] > 1.e-4).all())
    assert_array_almost_equal(F_sparse, F)
    assert_array_almost_equal(pv_sparse, pv)
    print("a")


In [ ]:
X = pandas.get_dummies(X)

#thresholder = VarianceThreshold(threshold=0.5)
#X = thresholder.fit_transform(X)
#X = GenericUnivariateSelect(f_classif, mode='percentile',param=5).fit(X, Y).transform(X)
selector = SelectKBest(f_classif, k=3)
selector.fit(X, Y)
X = selector.transform(X)

seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=10, random_state=seed)

In [ ]:
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
#results = model_selection.cross_val_score(model, X, Y, cv=kfold)
#print(results.mean())
# Create recursive feature eliminator that scores features by mean squared errors
estimator = SVR(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector = selector.fit(X, Y)
print(selector.support_)